In [ ]:
import ee
import geemap

ee.Initialize()

studyArea = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])

# Load Global Forest Change dataset and select relevant bands within the study area.
gfc = ee.Image('UMD/hansen/global_forest_change_2020_v1_8') \
        .select(['loss', 'gain', 'treecover2000']) \
        .clip(studyArea)

# Identify forested areas with more than 30% tree cover in 2000.
forestMask = gfc.select('treecover2000').gt(30)

# Apply forest mask to loss and gain to calculate tree cover change only within forested areas.
treeCoverLoss = gfc.select('loss').updateMask(forestMask)
treeCoverGain = gfc.select('gain').updateMask(forestMask)

# Convert binary loss and gain to area (hectares) by multiplying by pixel area and dividing by 10,000 (m² in a hectare).
lossArea = treeCoverLoss.multiply(ee.Image.pixelArea()).divide(10000)
gainArea = treeCoverGain.multiply(ee.Image.pixelArea()).divide(10000)

# Visualization and statistics using Jupyter Notebooks.
Map = geemap.Map()
Map.addLayer(lossArea, {'min': 0, 'max': 1, 'palette': ['FF0000']}, 'Tree Cover Loss')
Map.addLayer(gainArea, {'min': 0, 'max': 1, 'palette': ['00FF00']}, 'Tree Cover Gain')
Map

# Calculate statistics: total area of tree cover loss and gain within the study area.
statsLoss = lossArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=studyArea,
    scale=30,  # Consider adjusting scale for large areas
    maxPixels=1e9
)
statsGain = gainArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=studyArea,
    scale=30,  # Consider adjusting scale for large areas
    maxPixels=1e9
)

# print out statistics (without using getInfo for large datasets):
loss_area_ha = statsLoss.getInfo()['loss']
gain_area_ha = statsGain.getInfo()['gain']
print(f'Tree Cover Loss (ha): {loss_area_ha}')
print(f'Tree Cover Gain (ha): {gain_area_ha}')

This script is designed to analyze forest cover changes using the Google Earth Engine (GEE) platform with Python. It focuses on detecting areas of tree cover loss and gain within a specified study area based on the Global Forest Change dataset by Hansen et al. for the year 2020. The script follows these main steps:

1. **Initialization**: It begins by importing necessary libraries (`ee` for Earth Engine operations and `geemap` for visualization in a Jupyter Notebook environment) and initializing the Earth Engine API.

2. **Study Area Definition**: The study area is defined as a rectangle with given coordinates (xmin, ymin, xmax, ymax). These coordinates should be replaced with the actual coordinates of the desired study area.

3. **Data Loading and Selection**: The Global Forest Change dataset is loaded, and relevant bands (`loss`, `gain`, `treecover2000`) are selected and clipped to the study area. The dataset includes data on tree cover loss, gain, and baseline tree cover for the year 2000.

4. **Forest Mask Application**: A forest mask is created by thresholding the `treecover2000` band to identify areas with more than 30% tree cover, considering them as forested areas.

5. **Tree Cover Change Calculation**: The script calculates areas of tree cover loss and gain within the forested areas by applying the forest mask to the `loss` and `gain` bands. These binary masks are then converted to area measurements (in hectares) by multiplying by the pixel area and dividing by 10,000 (to convert square meters to hectares).

6. **Visualization**: Using `geemap`, the script adds layers for tree cover loss and gain to a map for visualization, with loss areas displayed in red (`FF0000`) and gain areas in green (`00FF00`).

7. **Statistics Calculation**: It calculates the total area of tree cover loss and gain within the study area using `reduceRegion` with a `sum` reducer. The scale and `maxPixels` parameters are set to accommodate large areas and ensure comprehensive coverage.

8. **Output**: Finally, the script prints the calculated total areas of tree cover loss and gain in hectares. Note that `getInfo()` is used for demonstration purposes; in operational scripts, especially those handling large datasets, alternative strategies may be more appropriate to avoid potential memory issues.

I think this script provides a powerful yet straightforward means of analyzing forest dynamics, leveraging satellite-based observations to support environmental monitoring, policy-making, and conservation efforts.

**REFERENCES**
- [Global Forest Change 2000–2020](https://storage.googleapis.com/earthenginepartners-hansen/GFC-2020-v1.8/download.html)